<a href="https://colab.research.google.com/github/muskanverma12/crop-detection-by-google-colab/blob/main/crop_hackathone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow keras


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input

model = Sequential([
    Input(shape=(224,224,3)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.save('dummy_crop_model.keras')
print("Model saved successfully!")


Model saved successfully!


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')


Saving leaf.png to leaf.png
User uploaded file "leaf.png" with length 150107 bytes


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('dummy_crop_model.keras')


img_path = list(uploaded.keys())[0]

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img) / 255.0


x = np.expand_dims(x, axis=0)


preds = model.predict(x)


class_names = ['Disease A', 'Disease B', 'Healthy']


predicted_class = class_names[np.argmax(preds)]

print("Prediction probabilities:", preds)
print("Predicted Class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
Prediction probabilities: [[0.3591536  0.4520737  0.18877263]]
Predicted Class: Disease B


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('dummy_crop_model.keras')
class_names = ['Disease A', 'Disease B', 'Healthy']

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    predicted_class = class_names[np.argmax(preds)]
    return preds, predicted_class

upload_widget = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

output = widgets.Output()

def on_upload_change(change):
    output.clear_output()
    for filename, fileinfo in upload_widget.value.items():
        with open(filename, 'wb') as f:
            f.write(fileinfo['content'])
        preds, predicted_class = predict_image(filename)
        with output:
            print(f'Prediction probabilities: {preds}')
            print(f'Predicted Class: {predicted_class}')

upload_widget.observe(on_upload_change, names='value')

display(upload_widget, output)


FileUpload(value={}, accept='image/*', description='Upload')

Output()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [ ]:

# !pip install tensorflow pillow ipywidgets

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from PIL import Image
import io
import time
import os

# Check and create dummy model if not present
if not os.path.exists('dummy_crop_model.keras'):
    model = keras.Sequential([
        layers.Input(shape=(224, 224, 3)),
        layers.Conv2D(16, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.save('dummy_crop_model.keras')
    print("✅ Dummy model created and saved.")
else:
    print("✅ Model already exists.")

# Load your saved model
model = tf.keras.models.load_model('dummy_crop_model.keras')
class_names = ['Mild Infection', 'Severe Infection', 'Healthy Leaf']

# Prediction function
def predict_image(img):
    img = img.resize((224, 224))
    img = img.convert('RGB')
    x = np.array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)[0]
    predicted_idx = np.argmax(preds)
    predicted_class = class_names[predicted_idx]
    confidence = preds[predicted_idx]
    return preds, predicted_class, confidence

# Widgets
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
img_preview = widgets.Image(format='png', width=280, height=280)
clear_button = widgets.Button(description="Clear", button_style='warning', icon='times')
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Processing:', style={'description_width': 'initial'})
output_html = widgets.HTML(value="")

# Upload event handler
def on_upload_change(change):
    output_html.value = ""
    progress_bar.value = 0
    if len(upload_widget.value) == 0:
        img_preview.value = b''
        return

    progress_bar.value = 10

    for filename, fileinfo in upload_widget.value.items():
        img_data = fileinfo['content']
        img_preview.value = img_data

        img = Image.open(io.BytesIO(img_data))

        for i in range(20, 100, 20):
            time.sleep(0.2)
            progress_bar.value = i

        preds, predicted_class, confidence = predict_image(img)

        # Build string for class probabilities
        probs_str = ""
        for i, class_name in enumerate(class_names):
            probs_str += f"<li><b>{class_name}:</b> {preds[i]*100:.2f}%</li>"

        # Leaf condition status in simple terms
        if predicted_class == "Healthy Leaf":
            leaf_status = "<span style='color:#4caf50; font-weight:bold;'>Leaf is fresh and healthy (No Disease)</span>"
        else:
            leaf_status = f"<span style='color:#e53935; font-weight:bold;'>Leaf is damaged or infected: {predicted_class}</span>"

        html_output = f"""
        <div style="
            font-family: Arial, sans-serif;
            border: 2px solid #888888;
            padding: 15px;
            border-radius: 8px;
            background: #222222;
            color: #eeeeee;
        ">
            <h3 style="color: #ffffff; margin-top: 0;">Prediction</h3>
            <p><b>Leaf Condition:</b> {leaf_status}</p>
            <p><b>Confidence Level:</b> <span style="color: #bbdefb;">I am {confidence*100:.2f}% sure about this.</span></p>
            <ul style="list-style:none; padding-left:0;">
                {probs_str}
            </ul>
        </div>
        """

        output_html.value = html_output
        progress_bar.value = 100

# Clear button event
def on_clear_click(b):
    upload_widget.value.clear()
    img_preview.value = b''
    progress_bar.value = 0
    output_html.value = ""

# Link events
upload_widget.observe(on_upload_change, names='value')
clear_button.on_click(on_clear_click)

# Display UI
display(widgets.VBox([
    widgets.Label("Upload a Leaf Image:", style={'font-weight': 'bold'}),
    upload_widget,
    img_preview,
    progress_bar,
    clear_button,
    output_html
]))


✅ Dummy model created and saved.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
